In [ ]:
from pathlib import Path

import duckdb as db
import numpy as np
import pandas as pd
import srsly
from duckdb.typing import DOUBLE
from tqdm.auto import tqdm

In [ ]:
run_path = Path("../outputs")

Export hparams

In [ ]:
# paths = list(run_path.rglob("*hparams.yaml"))
paths = [p for p in run_path.rglob("*hparams.yaml") if any(i in str(p) for i in ("ablations", "main", "other_models"))]

hparams = []
for p in tqdm(paths):
    h: dict = srsly.read_yaml(p)  # type: ignore
    h["filename"] = str(p.parents[0])
    hparams.append(h)

hparams_df = pd.concat([pd.json_normalize(h, sep=".") for h in hparams])

In [ ]:
hparams_df.to_csv("../results/hparams.tsv", sep="\t", index=False)

Export AUCs

In [ ]:
# paths = [str(p) for p in run_path.rglob("*tensorboard_logs.parquet")]
paths = [
    str(p)
    for p in run_path.rglob("*tensorboard_logs.parquet")
    if any(i in str(p) for i in ("ablations", "main", "other_models"))
]

In [ ]:
tbl = db.sql(
    f"""
select
    step, 
    tag, 
    value, 
    parse_dirpath(filename) as filename
from read_parquet({paths}, filename=True)
where contains(tag, 'test/f1_class') or tag == 'timer/query_time'
"""
)

In [ ]:
def fn(x: list[float]) -> float:
    return np.trapz(x)


# con.remove_function("trapz")
db.create_function("trapz", fn, ["DOUBLE[]"], DOUBLE)  # type: ignore

In [ ]:
res_tbl = db.sql(
    """
-- Collect values into list
with ctx as (
select
    * exclude (value), 
    list(value) over (partition by filename, tag order by step) as values
from tbl
)

-- Aggregate
select
    * exclude (values),
    case
        when contains(tag, 'timer/') then list_sum(values) / 60
        else trapz(values)
    end as value
from ctx
"""
)

In [ ]:
res_df = res_tbl.df()

In [ ]:
res_df.to_parquet("../results/metrics.parquet", index=False)

Export subpool information

In [ ]:
paths = [
    str(p)
    for p in run_path.rglob("*subpool_ids.jsonl")
    if any(i in str(p) for i in ("ablations", "main", "other_models"))
]

In [ ]:
subpool_tbl = db.sql(
    f"""
select
    parse_dirpath(parse_dirpath(filename)) as filename,
    labelling_round, 
    unnest(subpool_ids) as subpool_ids
from read_json({paths}, filename=True, columns = {{subpool_ids: 'INT32[]', labelling_round: 'INT32'}})
"""
)

In [ ]:
db.sql("copy subpool_tbl to '../results/subpool_ids.parquet' (format 'parquet', codec 'zstd')")

Export labelled dataset

In [ ]:
paths = [
    str(p)
    for p in run_path.rglob("*labelled_dataset.parquet")
    if any(i in str(p) for i in ("ablations", "main", "other_models"))
]

In [ ]:
tbl = db.sql(
    f"""
with ctx as (
    select
        parse_dirpath(parse_dirpath(filename)) as filename,
        uid,
        labels,
        labelling_round
    from read_parquet({paths}, filename=True)
    where is_labelled == true
)
select 
    filename,
    labelling_round,
    labels,
    count(1) as n
from ctx
group by filename, labelling_round, labels
"""
)

In [ ]:
label_tbl = db.sql(
    """
select *, sum(n) over (partition by filename, labels order by labelling_round) as cum_n
from tbl
"""
)

In [ ]:
db.sql("copy label_tbl to '../results/labelled_ids.parquet' (format 'parquet', codec 'zstd')")